In [1]:
import pandas as pd
import pyodbc
from sqlalchemy import create_engine
import mysql.connector
from datetime import datetime, timedelta

In [2]:
db_config = {
    'host': 'localhost',
    'user': 'root',
    'password': 'abcd1234',
    'database': 'iitr',
}
connection = mysql.connector.connect(**db_config)
cursor = connection.cursor()
engine = create_engine(f"mysql+mysqlconnector://{db_config['user']}:{db_config['password']}@{db_config['host']}/{db_config['database']}")

In [3]:
connection = mysql.connector.connect(**db_config)
cursor = connection.cursor()

# Loop through dates
start_date = "28082024"
end_date = "29082024"
dates = []
current_date = datetime.strptime(start_date, "%d%m%Y")
end_dt = datetime.strptime(end_date, "%d%m%Y")

while current_date <= end_dt:
    dates.append(current_date.strftime("%d%m%Y"))
    current_date += timedelta(days=1)

for date in dates:
    day_of_week = datetime.strptime(date, "%d%m%Y").weekday()
    if day_of_week in [5, 6]:
        continue

    # Read CSV data
    try:
        data = pd.read_csv(f"/Users/nandiniupadhyay/Desktop/IIT Roorkee/Downloading Bhavcopy2024/bhav{date}.csv")
    except FileNotFoundError:
        continue

    # Adjust date format if necessary
    data['DATE1'] = pd.to_datetime(data['DATE1'], format='%d-%b-%Y').dt.strftime('%Y-%m-%d')

    # Insert data into MySQL using cursor
    for index, row in data.iterrows():
        try:
            cursor.execute("""
                INSERT INTO stockdata2024 (SYMBOL, SERIES, DATE1, PREV_CLOSE, OPEN_PRICE, HIGH_PRICE, LOW_PRICE, LAST_PRICE, CLOSE_PRICE, AVG_PRICE, TTL_TRD_QNTY, TURNOVER_LACS, NO_OF_TRADES, DELIV_QTY, DELIV_PER)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """, tuple(row))
            connection.commit()  # Commit after each row insertion
        except mysql.connector.Error as err:
            print(f"Error inserting row: {err}")
            connection.rollback()  # Rollback the transaction if there's an error

# Close cursor and connection
cursor.close()
connection.close()